In [345]:
import tweepy


ModuleNotFoundError: No module named 'tweepy'

In [2]:
import urllib3
import urllib3.request

import csv
import json
from pandas.io.json import json_normalize

import pandas as pd
import numpy as np

In [327]:
pd.options.display.max_colwidth = 1000


In [344]:
####input your credentials here
consumer_key = '4ha4rLgP6Ci6fEZtaqttGTKoA'
consumer_secret = '5ckLaCgfTdfmWM7qS9f2w05pDCSIWRCTHlm7RLnKwK9tCWIz9P'
access_token = '602145669-jHmxtsl0wSZDFeZxi81GcTzYrD87dRBhF78ip0qo'
access_token_secret = 'YFLMmVVdcN4gb4KDX3MeOjbjxoKnnsFvjKxjRGMkkEZ5D'

In [9]:
input_hashtag = "giletjaune"
n_tweets = 1
language = "fr"
since_date = "2018-03-07"

In [343]:
####request to Twitter API
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth,wait_on_rate_limit=True,wait_on_rate_limit_notify = True)

inc = 0
for tweet in tweepy.Cursor(api.search,q="#"+input_hashtag,#count=n_tweets,
                           lang=language,
                           since=since_date, tweet_mode=extended).items():
    inc = +1
    with open('tweets/tweet'+str(tweet.id)+'.json', 'w', encoding='utf8') as file:
        ####you need to create a 'tweets' folder
        
        tweet_json = tweet._json
        json.dump(tweet_json, file)
        

NameError: name 'tweepy' is not defined

In [4]:
def is_a_retweet(tweet):
    #### returns a boolean 
    return tweet.get('retweeted_status',None) != None

In [295]:
#initialisation of databases, df files

d_users = {'user_id': [],
           'name':[],
           'screen_name':[],
           'location':[],
           'description':[],
           'url':[],
           'followers_count':[],
           'friends_count':[],
           'listed_count':[],
           'created_at':[],
           'favourites_count':[],
           'geo_enabled':[],
           'verified':[],
           'statuses_count':[],
           'lang':[],
           'contributors_enabled':[]
          }
df_users = pd.DataFrame(data=d_users)

d_tweets = {'tweet_id': [],
            'created_at': [],
            'text': [],
            'truncated': [],
            'source': [],
            'in_reply_to_status_id': [],
            'in_reply_to_user_id': [],
            'in_reply_to_screen_name': [],
            'user_id': [],
            'geo': [],
            'coordinates': [],
            'place': [],
            'contributors': [],
            'is_quote_status': [],
            'retweet_count': [],
            'favorite_count': [],
            'favorited': [],
            'retweeted': [],
            'lang': []         
           }
df_tweets = pd.DataFrame(data=d_tweets)


d_hashtags={'hashtag': [], 'tweet_id': []}
df_hashtags = pd.DataFrame(data=d_hashtags)

d_users_mentions = {'tweet_id':[], 'user_id': [], 'screen_name':[], 'name':[]}
df_users_mentions = pd.DataFrame(data=d_users_mentions)

d_retweet_users = {'user_id':[], 'original_user_id':[], 'original_tweet_id':[]}
df_retweet_users= pd.DataFrame(data=d_retweet_users)

d_influent_users = {'user_id':[], 
                    'name':[],
                    'retweet_count':[],
                    'favorite_count':[], 
                    'description':[] 
                   }

df_influent_users = pd.DataFrame(data=d_influent_users)



In [296]:
def store_hashtag(tweet):
    global input_hashtag
    global df_hashtags
    if tweet['entities']['hashtags']==[]:
        df_hashtags = df_hashtags.append({'hashtag': input_hashtag ,'tweet_id':tweet['id_str']}, ignore_index=True)
    else:
        for raw_hash in tweet['entities']['hashtags']:
            df_hashtags = df_hashtags.append({'hashtag': raw_hash['text'] ,'tweet_id':tweet['id_str']}, ignore_index=True)

def store_user_mentions(tweet):
    global df_users_mentions
    tweet_id = tweet['id_str']
    for raw_mention in tweet['entities']['user_mentions']:
        screen_name = raw_mention['screen_name']
        name = raw_mention['name']
        user_id = raw_mention['id_str']
        df_users_mentions = df_users_mentions.append({'tweet_id':tweet_id, 'user_id': user_id, 'screen_name':screen_name, 'name':name},ignore_index=True)

def store_users(tweet):
    global df_users
    user = tweet['user']
    if user['id_str'] not in df_users['user_id'].values:
        
        d_user = {'user_id': user['id_str'],
                  'name':user['name'],
                  'screen_name':user['screen_name'],
                  'location':user['location'],
                  'description':user['description'],
                  'url':user['url'],
                  'followers_count':user['followers_count'],
                  'friends_count':user['friends_count'],
                  'listed_count':user['listed_count'],
                  'created_at':user['created_at'],
                  'favourites_count':user['favourites_count'],
                  'geo_enabled':user['geo_enabled'],
                  'verified':user['verified'],
                  'statuses_count':user['statuses_count'],
                  'lang':user['lang'],
                  'contributors_enabled':user['contributors_enabled']
                 }
        df_users = df_users.append(d_user,ignore_index=True)

def store_tweet(tweet):
    global df_tweets
    
    df_tweets = df_tweets.append({'tweet_id':  tweet['id_str'],
            'created_at': tweet['created_at'],
            'text': tweet['text'],
            'truncated': tweet['truncated'],
            'source': tweet['source'],
            'in_reply_to_status_id': tweet['in_reply_to_status_id_str'],
            'in_reply_to_user_id': tweet['in_reply_to_user_id_str'],
            'in_reply_to_screen_name': tweet['in_reply_to_screen_name'],
            'user_id': tweet['user']['id_str'],
            'geo': tweet['geo'],
            'coordinates': tweet['coordinates'],
            'place': tweet['place'],
            'contributors': tweet['contributors'],
            'is_quote_status': tweet['is_quote_status'],
            'retweet_count': tweet['retweet_count'],
            'favorite_count': tweet['favorite_count'],
            'favorited': tweet['favorited'],
            'retweeted': tweet['retweeted'],
            'lang': tweet['lang'] },ignore_index=True)
        
def store_retweet_user(tweet):
    global df_retweet_users
    if is_a_retweet(tweet):
        df_retweet_users = df_retweet_users.append({'user_id':tweet['user']['id_str'],
                                                    'original_user_id':tweet['retweeted_status']['user']['id_str'],
                                                    'original_tweet_id':tweet['retweeted_status']['id_str']}, ignore_index=True)
        
def store_influent_users(n):
    #n is the number of influent users return
    df_favorite_count = df_tweets.groupby(['user_id'])['favorite_count'].agg('sum').sort_values(ascending=False).head()
    for i in range(n):
        d_influent_users = {'user_id':df_favorite_count[0], 
                            'name':[],
                            'retweet_count':[],
                            'favorite_count':[], 
                            'description':[] 
                           }
        

In [298]:
#### creation of the databases

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning) #https://stackoverflow.com/questions/40659212/futurewarning-elementwise-comparison-failed-returning-scalar-but-in-the-futur
import os
path_to_json = 'tweets/'
json_files = [pos_json for pos_json in os.listdir(path_to_json) if pos_json.endswith('.json')]
for json_file in json_files:
    
    with open("tweets/"+json_file) as tweet:
        tweet = json.load(tweet)
        if is_a_retweet(tweet):
            store_retweet_user(tweet)
        else:
            store_tweet(tweet)
            store_users(tweet)
            store_hashtag(tweet)
            store_user_mentions(tweet)
        
        
        

In [289]:
df_tweets.head(10);


In [290]:
df_tweets['text'].iloc[0]

'Parfaitement résumé , \n Quel clarté,je valide totalement 🏆🏆🏆🏆🏆\n#giletjaune #reseauxsociaux \n#media #Cnews #BFM… https://t.co/Bx5zEnxFL6'

In [299]:
#primary keys are ok?
print(df_tweets.duplicated('tweet_id').any())
print(df_users.duplicated('user_id').any())


False
False


# Aggregations


## Top users

In [205]:
df_most_favorited_users = df_tweets.groupby(['user_id'])['favorite_count'].agg('sum').sort_values(ascending=False).reset_index().head(15)
df_most_retweeted_users = df_tweets.groupby(['user_id'])['retweet_count'].agg('sum').sort_values(ascending=False).reset_index().head(15)
df_most_favorited_users['favorite_count'] = df_most_favorited_users['favorite_count'].astype('int32')
df_most_retweeted_users['retweet_count'] = df_most_retweeted_users['retweet_count'].astype('int32')

df_most_favorited_users['user_id'] = df_most_favorited_users['user_id'].astype(object)
df_most_retweeted_users['user_id'] = df_most_retweeted_users['user_id'].astype(object)



In [206]:
df_most_favorited_users_with_info = df_most_favorited_users.set_index('user_id').join(df_users.set_index('user_id')).drop_duplicates().sort_values('favorite_count',ascending=False)
df_most_retweeted_users_with_info = df_most_retweeted_users.set_index('user_id').join(df_users.set_index('user_id')).drop_duplicates().sort_values('retweet_count',ascending=False)

In [207]:
df_most_retweeted_users_with_info.head(15);

In [208]:
df_most_favorited_users_with_info.head(15);

In [209]:
df_most_retweeted_users_with_info.reset_index()
df_most_favorited_users_with_info.reset_index();

In [210]:
print(df_most_retweeted_users_with_info['name'])
print(df_most_favorited_users_with_info['name'])

user_id
1028668294751039488            LesGiletsJaunes.fr
338955445                     de Cabarrus Thierry
1095041610709172225               yellow defender
155606403                          MichaelVllt 🇫🇷
996698079180050433     ⚡️P'tweet Revue de Presse🗞
26110930                                      LCI
1064870090028527616             Rack.auto ❤🌍☀🚗🚌🏍🗽
3303607199                          arnaud hachet
1033669001526829056                        SMAGMA
33962372                           Ch.LECHEVALIER
15339119                             Fred Softway
367916454                          Duval Philippe
2458066446                       Alex_PatrioteGPI
2545538228                             MISS_STRAL
1102136594994221056                  Éteint Ta Tv
Name: name, dtype: object
user_id
1028668294751039488            LesGiletsJaunes.fr
338955445                     de Cabarrus Thierry
1064870090028527616             Rack.auto ❤🌍☀🚗🚌🏍🗽
26110930                                      LCI
33962372

In [211]:
df_tweets_by_user = df_tweets.groupby(['user_id']).size().to_frame().reset_index()
df_most_favorited_users_with_info = df_most_favorited_users_with_info.join(df_tweets_by_user.set_index('user_id')).drop_duplicates().sort_values('favorite_count',ascending=False)
df_most_retweeted_users_with_info = df_most_retweeted_users_with_info.join(df_tweets_by_user.set_index('user_id')).drop_duplicates().sort_values('retweet_count',ascending=False)

In [212]:
df_most_favorited_users_with_info;

In [239]:
top_influencers_fav = df_most_favorited_users_with_info[['name','description', 'followers_count', 'friends_count','favorite_count',0]]
top_influencers_rt = df_most_retweeted_users_with_info[['name','description', 'followers_count', 'friends_count','retweet_count',0]]
top_influencers_fav.columns = ["Name","Description","Number of followers","Number of friends","Number of favorites","Number of tweets"]
top_influencers_rt.columns = ["Name","Description","Number of followers","Number of friends","Number of retweets","Number of tweets"]
top_influencers_fav = top_influencers_fav.reset_index().drop(columns = ["user_id"])
top_influencers_rt = top_influencers_rt.reset_index().drop(columns = ["user_id"]);



In [240]:
#add ratios
top_influencers_fav.loc[:,'Like Tweet Ratio'] = top_influencers_fav.loc[:,"Number of favorites"]/top_influencers_fav.loc[:,"Number of tweets"]
top_influencers_rt.loc[:,'Retweet Tweet Ratio'] = top_influencers_rt.loc[:,"Number of retweets"]/top_influencers_rt.loc[:,"Number of tweets"]

In [220]:
top_influencers_fav

,Name,Description,Number of followers,Number of friends,Number of favorites,Number of tweets,Like Tweet Ratio
user_id,,,,,,,
1028668294751039488,LesGiletsJaunes.fr,Mouvement citoyen apartisan pour une démocratie d'avenir. #GiletJaune #GiletsJaunes #RIC,28518,0,1618,6,270
338955445,de Cabarrus Thierry,"Après 44 ans de journalisme ""objectif"" (carte de presse N°35183), je reprends ma liberté et j'abandonne une neutralité confortable. Ex chroniqueur sur Le Plus",7983,1042,53,3,18
1064870090028527616,Rack.auto ❤🌍☀🚗🚌🏍🗽,"Macho idéal, évaluateur précoce, apolitique, athée, tous : partis, religions, couleurs. Contre : les causes injustes, le sectarisme, l’égalité à sens unique.",569,1966,31,11,3
26110930,LCI,"Bienvenue sur LCI, #La26 sur la TNT. Le meilleur de l'info, de l'analyse, du décryptage et tous les grands événements en direct #RespectZone",151892,994,31,3,10
33962372,Ch.LECHEVALIER,"Gaulliste, ex-RPR / DLR, avec Marine en Ille & Vilaine. Conseiller Régional BzH, Pr Agrégé ; Ancien Elève de l'EN3S ; Licencié en Histoire, DEA de Droit Civil",12494,8994,23,2,12
155606403,MichaelVllt 🇫🇷,Rebel with many causes. https://t.co/ISNLNObWbX,7338,991,22,1,22
1095041610709172225,yellow defender,Patriote et \nGILET JAUNE.\nAntin macron,21,20,21,3,7
3303607199,arnaud hachet,De la lumière s'il vous plait,193,360,18,7,3
2569469946,Ptit-eOurs-eBlanc-he,PtitOursBlanc (rien à voir avec les fachos!)\nPr les curieux : \nhttps://t.co/46wHWjKdyF…\n\n#injustices #AntiHaine #LaPlaneteVaMal,346,795,15,2,8


## Top hashtags for this input hashtags and by users

In [347]:
top_influencers_fav.style.set_properties(**{'text-align': 'left'});

In [351]:
df_tweets_hashtags

,hashtag,created_at,text,truncated,source,in_reply_to_status_id,in_reply_to_user_id,in_reply_to_screen_name,user_id,geo,coordinates,place,contributors,is_quote_status,retweet_count,favorite_count,favorited,retweeted,lang
tweet_id,,,,,,,,,,,,,,,,,,,
1105895180341837824,giletjaune,Wed Mar 13 18:15:32 +0000 2019,"Parfaitement résumé , \n Quel clarté,je valide totalement 🏆🏆🏆🏆🏆\n#giletjaune #reseauxsociaux \n#media #Cnews #BFM… https://t.co/Bx5zEnxFL6",1,"<a href=""http://twitter.com/download/android"" rel=""nofollow"">Twitter for Android</a>",None,None,None,2569469946,None,None,None,None,0,8,15,,,fr
1105895180341837824,reseauxsociaux,Wed Mar 13 18:15:32 +0000 2019,"Parfaitement résumé , \n Quel clarté,je valide totalement 🏆🏆🏆🏆🏆\n#giletjaune #reseauxsociaux \n#media #Cnews #BFM… https://t.co/Bx5zEnxFL6",1,"<a href=""http://twitter.com/download/android"" rel=""nofollow"">Twitter for Android</a>",None,None,None,2569469946,None,None,None,None,0,8,15,,,fr
1105895180341837824,media,Wed Mar 13 18:15:32 +0000 2019,"Parfaitement résumé , \n Quel clarté,je valide totalement 🏆🏆🏆🏆🏆\n#giletjaune #reseauxsociaux \n#media #Cnews #BFM… https://t.co/Bx5zEnxFL6",1,"<a href=""http://twitter.com/download/android"" rel=""nofollow"">Twitter for Android</a>",None,None,None,2569469946,None,None,None,None,0,8,15,,,fr
1105895180341837824,Cnews,Wed Mar 13 18:15:32 +0000 2019,"Parfaitement résumé , \n Quel clarté,je valide totalement 🏆🏆🏆🏆🏆\n#giletjaune #reseauxsociaux \n#media #Cnews #BFM… https://t.co/Bx5zEnxFL6",1,"<a href=""http://twitter.com/download/android"" rel=""nofollow"">Twitter for Android</a>",None,None,None,2569469946,None,None,None,None,0,8,15,,,fr
1105895180341837824,BFM,Wed Mar 13 18:15:32 +0000 2019,"Parfaitement résumé , \n Quel clarté,je valide totalement 🏆🏆🏆🏆🏆\n#giletjaune #reseauxsociaux \n#media #Cnews #BFM… https://t.co/Bx5zEnxFL6",1,"<a href=""http://twitter.com/download/android"" rel=""nofollow"">Twitter for Android</a>",None,None,None,2569469946,None,None,None,None,0,8,15,,,fr
1105896672536784896,GiletsJaunes,Wed Mar 13 18:21:27 +0000 2019,#GiletsJaunes \n#GiletsJaunesParis \n#GiletsJaunesEU \n#TousDansLaRue16mars\n#FemmesGiletsJaunes \n#Giletjaune… https://t.co/M4ZBNNJnYH,1,"<a href=""http://twitter.com/download/android"" rel=""nofollow"">Twitter for Android</a>",None,None,None,806828978342424576,None,None,None,None,1,3,1,,,fr
1105896672536784896,GiletsJaunesParis,Wed Mar 13 18:21:27 +0000 2019,#GiletsJaunes \n#GiletsJaunesParis \n#GiletsJaunesEU \n#TousDansLaRue16mars\n#FemmesGiletsJaunes \n#Giletjaune… https://t.co/M4ZBNNJnYH,1,"<a href=""http://twitter.com/download/android"" rel=""nofollow"">Twitter for Android</a>",None,None,None,806828978342424576,None,None,None,None,1,3,1,,,fr
1105896672536784896,GiletsJaunesEU,Wed Mar 13 18:21:27 +0000 2019,#GiletsJaunes \n#GiletsJaunesParis \n#GiletsJaunesEU \n#TousDansLaRue16mars\n#FemmesGiletsJaunes \n#Giletjaune… https://t.co/M4ZBNNJnYH,1,"<a href=""http://twitter.com/download/android"" rel=""nofollow"">Twitter for Android</a>",None,None,None,806828978342424576,None,None,None,None,1,3,1,,,fr
1105896672536784896,TousDansLaRue16mars,Wed Mar 13 18:21:27 +0000 2019,#GiletsJaunes \n#GiletsJaunesParis \n#GiletsJaunesEU \n#TousDansLaRue16mars\n#FemmesGiletsJaunes \n#Giletjaune… https://t.co/M4ZBNNJnYH,1,"<a href=""http://twitter.com/download/android"" rel=""nofollow"">Twitter for Android</a>",None,None,None,806828978342424576,None,None,None,None,1,3,1,,,fr


In [350]:
df_tweets_hashtags = df_hashtags.set_index('tweet_id').join(df_tweets.set_index('tweet_id'))

In [352]:
df_tw_hash_user = df_tweets_hashtags.set_index('user_id').join(df_users.drop(columns=['created_at', 'lang']).set_index('user_id'))

In [354]:
#df_aggretated = df_tw_hash_user.groupby(['user_id','hashtag']).size().reset_index()
df_aggretated = df_tw_hash_user.groupby(['name','hashtag']).size().reset_index()



In [369]:
df_aggretated.columns = ["Name", "#", "Used"]
df_aggretated;


In [370]:
df_aggretated[df_aggretated['Name'] == 'A-Boréal'][['#',"Used"]].sort_values("Used", ascending = False)

,#,Used
12,Européenne,3
9,Brexit,3
22,Pays,2
20,NouvelleZelande,2
18,Mascaxit,2
26,SachezLe,2
24,RT,2
11,Définition,2
10,Dissuader,2
35,pays,1


In [335]:
#useful encapsulation, aint it?
def get_tweet_by_userid(user_id):
    df_tweets[df_tweets['user_id'] == 'user_id']

def get_tweet_by_username(name):
    pd.options.display.max_colwidth = 1000
    df_tweets_user_info = df_tweets.set_index('user_id').join(df_users.drop(columns=['created_at', 'lang']).set_index('user_id'))
    df_tweets_user_info = df_tweets_user_info[df_tweets_user_info['name'] == 'LesGiletsJaunes.fr']
    df_tweets_user_info = df_tweets_user_info[['tweet_id','text','created_at', 'favorite_count', 'retweet_count']]
    return df_tweets_user_info

In [336]:
get_tweet_by_username('LesGiletsJaunes.fr')


,tweet_id,text,created_at,favorite_count,retweet_count
user_id,,,,,
1028668294751039488,1106179848278626304,"Peut-être un jour , certains hommes se mettront-ils à penser par eux-mêmes ... ou resteront-ils à jamais formatés a… https://t.co/Idelg87SM1",Thu Mar 14 13:06:42 +0000 2019,61,41
1028668294751039488,1106185627102527488,"82 ans ... ""On souhaite que ce gouvernement et ce président s'en aillent, c'est notre seul objectif, parce que c'es… https://t.co/8frUbg1eMO",Thu Mar 14 13:29:40 +0000 2019,1377,954
1028668294751039488,1106224548444295168,"Chaos social, enfumage médiatique, privatisation des recettes, privatisation du pays par la dette.… https://t.co/HhTWIkRCN8",Thu Mar 14 16:04:19 +0000 2019,98,68
1028668294751039488,1106261424265932806,"Canada, les gilets jaunes de l'Alberta se rassemblent pour les pipelines et contre Notley | Sheila Gunn Reid… https://t.co/vVDmM8Omya",Thu Mar 14 18:30:51 +0000 2019,22,14
1028668294751039488,1106510464282968064,"Le capitalisme nous fait croire que l'on peut d'un coté exploiter l'homme et l'ensemble des ressources , détruire l… https://t.co/j4NHK09Vz0",Fri Mar 15 11:00:27 +0000 2019,38,31
1028668294751039488,1106643656197488640,"Respect de l'homme, de l'environnement, de la planète, ne font qu'un . Ne comptons pas sur le capitalisme, exploite… https://t.co/SJ82yEdcog",Fri Mar 15 19:49:42 +0000 2019,22,10


In [342]:
df_tweets[df_tweets['tweet_id'] == '1106179848278626304']['text']

86    Peut-être un jour , certains hommes se mettront-ils à penser par eux-mêmes ... ou resteront-ils à jamais formatés a… https://t.co/Idelg87SM1
Name: text, dtype: object

In [358]:
df_hashtags[df_hashtags['tweet_id'] == '1106179848278626304']

,hashtag,tweet_id
183,giletjaune,1106179848278626304


In [329]:
df_tweets_user_info

,text,created_at,favorite_count,retweet_count
user_id,,,,
1028668294751039488,"Peut-être un jour , certains hommes se mettront-ils à penser par eux-mêmes ... ou resteront-ils à jamais formatés a… https://t.co/Idelg87SM1",Thu Mar 14 13:06:42 +0000 2019,61,41
1028668294751039488,"82 ans ... ""On souhaite que ce gouvernement et ce président s'en aillent, c'est notre seul objectif, parce que c'es… https://t.co/8frUbg1eMO",Thu Mar 14 13:29:40 +0000 2019,1377,954
1028668294751039488,"Chaos social, enfumage médiatique, privatisation des recettes, privatisation du pays par la dette.… https://t.co/HhTWIkRCN8",Thu Mar 14 16:04:19 +0000 2019,98,68
1028668294751039488,"Canada, les gilets jaunes de l'Alberta se rassemblent pour les pipelines et contre Notley | Sheila Gunn Reid… https://t.co/vVDmM8Omya",Thu Mar 14 18:30:51 +0000 2019,22,14
1028668294751039488,"Le capitalisme nous fait croire que l'on peut d'un coté exploiter l'homme et l'ensemble des ressources , détruire l… https://t.co/j4NHK09Vz0",Fri Mar 15 11:00:27 +0000 2019,38,31
1028668294751039488,"Respect de l'homme, de l'environnement, de la planète, ne font qu'un . Ne comptons pas sur le capitalisme, exploite… https://t.co/SJ82yEdcog",Fri Mar 15 19:49:42 +0000 2019,22,10
